<a href="https://colab.research.google.com/github/VIJAYSAGAR-ALURI/Sentimental-Analysis-Of-Twitter-Data-Using-RNN/blob/main/Sentimental_Analysis_of_Tweets_Using__RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
csv_path = "/content/Tweets.csv"

try:
    df = pd.read_csv(csv_path, encoding='utf-8', engine='python', on_bad_lines='skip')
except UnicodeDecodeError:
    df = pd.read_csv(csv_path, encoding='ISO-8859-1', engine='python', on_bad_lines='skip')

if all(isinstance(col, int) for col in df.columns):
    if df.shape[1] >= 6:
        df.columns = ["target", "id", "date", "flag", "user", "text"]
    else:
        df.columns = [f"col_{i}" for i in range(df.shape[1])]

print("✅ Columns in dataset:", df.columns.tolist())

✅ Columns in dataset: ['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence', 'negativereason', 'negativereason_confidence', 'airline', 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone']


In [ ]:
def detect_text_column(df):
    text_candidates = ["text", "tweet", "review", "comment", "message", "content", "body"]
    for c in text_candidates:
        if c in df.columns:
            return c
    for col in df.columns:
        if df[col].dtype == 'object' and df[col].astype(str).str.len().mean() > 20:
            return col
    return df.columns[-1]

def detect_label_column(df):
    label_candidates = ["sentiment", "label", "emotion", "target", "category", "airline_sentiment", "polarity"]
    for c in label_candidates:
        if c in df.columns:
            return c
    for col in df.columns:
        if df[col].dtype == 'object' and df[col].nunique() < 20:
            return col
    return df.columns[0]

TEXT_COLUMN = detect_text_column(df)
LABEL_COLUMN = detect_label_column(df)

print(f"✅ Using Text Column: {TEXT_COLUMN}")
print(f"✅ Using Label Column: {LABEL_COLUMN}")


✅ Using Text Column: text
✅ Using Label Column: airline_sentiment


In [ ]:
stop_words = {
    "a","an","the","and","or","is","am","are","was","were","in","on","at","to","of","for","this","that","it","be","as",
    "with","by","from","but","if","then","so","too","very","can","will","just","do","does","did","have","has","had","not",
    "no","yes","you","i","me","my","we","us","our","they","them","their","he","she","his","her"
}

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+|#", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = " ".join([w for w in text.split() if w not in stop_words])
    return text.strip()

df["clean_text"] = df[TEXT_COLUMN].apply(clean_text)


In [ ]:
label_mapping = {
    "positive": "Happy", "negative": "Sad", "neutral": "Neutral",
    "angry": "Angry", "frustrated": "Angry", "excited": "Excited",
    "surprised": "Surprised", "relaxed": "Relaxed", "disgusted": "Disgusted",
    "fear": "Fearful", "joy": "Happy", "sadness": "Sad", "happiness": "Happy",
    "anger": "Angry", "0": "Sad", "1": "Neutral", "2": "Happy", "4": "Happy"
}

unique_labels = df[LABEL_COLUMN].astype(str).str.lower().unique()
for lbl in unique_labels:
    if lbl not in label_mapping:
        label_mapping[lbl] = lbl.capitalize()

df["emotion"] = df[LABEL_COLUMN].astype(str).str.lower().map(label_mapping).fillna("Neutral")

In [ ]:
le = LabelEncoder()
df["label_enc"] = le.fit_transform(df["emotion"])
num_classes = len(le.classes_)
print("✅ Detected Emotion Classes:", le.classes_)

✅ Detected Emotion Classes: ['Angry' 'Disgusted' 'Excited' 'Happy' 'Neutral' 'Relaxed' 'Sad'
 'Surprised']


In [ ]:
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 80

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(df["clean_text"])

X = tokenizer.texts_to_sequences(df["clean_text"])
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
y = df["label_enc"].values


In [ ]:
if len(np.unique(y)) > 1:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
else:
    X_train, X_test, y_train, y_test = X, X, y, y


In [ ]:
model = Sequential([
    Embedding(MAX_VOCAB_SIZE, 128, input_length=MAX_SEQUENCE_LENGTH),
    SimpleRNN(256, activation='tanh'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=64, verbose=2, callbacks=[es])

Epoch 1/10
149/149 - 2s - 12ms/step - accuracy: 0.9629 - loss: 0.1211 - val_accuracy: 0.7155 - val_loss: 1.4599
Epoch 2/10
149/149 - 2s - 11ms/step - accuracy: 0.9717 - loss: 0.0993 - val_accuracy: 0.7104 - val_loss: 1.6278
Epoch 3/10
149/149 - 2s - 11ms/step - accuracy: 0.9670 - loss: 0.1118 - val_accuracy: 0.7218 - val_loss: 1.4572
Epoch 4/10
149/149 - 2s - 12ms/step - accuracy: 0.9761 - loss: 0.0784 - val_accuracy: 0.7272 - val_loss: 1.4669
Epoch 5/10
149/149 - 2s - 10ms/step - accuracy: 0.9725 - loss: 0.0897 - val_accuracy: 0.6932 - val_loss: 1.4692
Epoch 6/10
149/149 - 2s - 10ms/step - accuracy: 0.9748 - loss: 0.0795 - val_accuracy: 0.7063 - val_loss: 1.5924


In [ ]:
pred_probs = model.predict(X, verbose=0)
pred_classes = np.argmax(pred_probs, axis=1)

df["Predicted_Emotion"] = [le.classes_[i] for i in pred_classes]
df["Confidence"] = np.max(pred_probs, axis=1)

def map_sentiment(e):
    if e in ["Happy", "Excited", "Relaxed", "Joyful"]:
        return "Positive"
    elif e in ["Sad", "Angry", "Disgusted", "Fearful"]:
        return "Negative"
    else:
        return "Neutral"

df["Sentiment"] = df["Predicted_Emotion"].apply(map_sentiment)

suggestions = {
    "Happy": "Keep spreading positivity!",
    "Neutral": "Stay calm and observe.",
    "Sad": "It's okay to feel sad sometimes. Talk to someone.",
    "Angry": "Try taking deep breaths. Calm helps.",
    "Excited": "Amazing! Use your energy wisely.",
    "Surprised": "Wow! Stay curious.",
    "Relaxed": "Peaceful and calm—keep it up.",
    "Disgusted": "Step away and do something you enjoy.",
    "Fearful": "Remember, courage grows through action."
}

df["Suggestion"] = df["Predicted_Emotion"].map(lambda e: suggestions.get(e, "Take care of yourself."))

In [ ]:
print("\n===============================")
print("🎯 SAMPLE PREDICTIONS FROM DATASET:")
print("===============================\n")
print(df[[TEXT_COLUMN, "emotion", "Predicted_Emotion", "Sentiment", "Confidence", "Suggestion"]].head(10))


🎯 SAMPLE PREDICTIONS FROM DATASET:

                                                text  emotion  \
0                @VirginAmerica What @dhepburn said.  Neutral   
1  @VirginAmerica plus you've added commercials t...    Happy   
2  @VirginAmerica I didn't today... Must mean I n...  Neutral   
3  @VirginAmerica it's really aggressive to blast...      Sad   
4  @VirginAmerica and it's a really big bad thing...      Sad   
5  @VirginAmerica seriously would pay $30 a fligh...      Sad   
6  @VirginAmerica yes, nearly every time I fly VX...    Happy   
7  @VirginAmerica Really missed a prime opportuni...  Neutral   
8    @virginamerica Well, I didn't…but NOW I DO! :-D    Happy   
9  @VirginAmerica it was amazing, and arrived an ...    Happy   

  Predicted_Emotion Sentiment  Confidence  \
0           Neutral   Neutral    0.756450   
1             Happy  Positive    0.999325   
2           Neutral   Neutral    0.983795   
3               Sad  Negative    0.999954   
4               Sad  N

In [ ]:
output_path = "/content/Predicted_Output.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Results saved to: {output_path}")


✅ Results saved to: /content/Predicted_Output.csv


In [ ]:
POS_WORDS = {"happy", "joy", "pleased", "delighted", "glad", "awesome", "great", "amazing", "love"}
NEG_WORDS = {"sad", "unhappy", "depressed", "miserable", "hate", "angry", "upset", "terrible", "bad"}

def contains_negation_before(word, text, window=3):
    tokens = text.split()
    try:
        idx = tokens.index(word)
    except ValueError:
        return False
    start = max(0, idx - window)
    context = tokens[start:idx]
    for w in context:
        if w in {"not", "no", "never", "n't"}:
            return True
    return False

def rule_based_prediction(cleaned):
    tokens = cleaned.split()
    for t in tokens:
        if t in POS_WORDS:
            if contains_negation_before(t, cleaned):
                return "Sad", "Negative", 0.9
            return "Happy", "Positive", 0.95
        if t in NEG_WORDS:
            if contains_negation_before(t, cleaned):
                return "Happy", "Positive", 0.9
            return "Sad", "Negative", 0.95
    return None

CONF_THRESH = 0.65

while True:
    user_input = input("\n💬 Enter a sentence to analyze sentiment (or type 'exit' to stop): ").strip()
    if user_input.lower() == 'exit':
        print("👋 Exiting sentiment analysis.")
        break

    cleaned_input = clean_text(user_input)
    if not cleaned_input:
        print("⚠️ Please enter a valid sentence.")
        continue

    seq = tokenizer.texts_to_sequences([cleaned_input])
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)

    pred = model.predict(padded, verbose=0)[0]
    pred_class_idx = int(np.argmax(pred))
    pred_class = le.classes_[pred_class_idx]
    confidence = float(pred[pred_class_idx])

    # Apply rule-based correction only if confidence is low
    if confidence < CONF_THRESH:
        rule = rule_based_prediction(cleaned_input)
        if rule:
            emotion, sentiment, confidence = rule
        else:
            emotion = pred_class
            sentiment = map_sentiment(pred_class)
    else:
        emotion = pred_class
        sentiment = map_sentiment(pred_class)

    suggestion = suggestions.get(emotion, "Take care of yourself.")

    print("\n===============================")
    print(f"🧠 Emotion Detected: {emotion}")
    print(f"💭 Sentiment: {sentiment}")
    print(f"📊 Confidence: {confidence:.2f}")
    print(f"💡 Suggestion: {suggestion}")
    print("===============================\n")



💬 Enter a sentence to analyze sentiment (or type 'exit' to stop): hi

🧠 Emotion Detected: Neutral
💭 Sentiment: Neutral
📊 Confidence: 0.98
💡 Suggestion: Stay calm and observe.


💬 Enter a sentence to analyze sentiment (or type 'exit' to stop): I am happy to meet my friends, but sad to go to college to meet them.

🧠 Emotion Detected: Neutral
💭 Sentiment: Neutral
📊 Confidence: 1.00
💡 Suggestion: Stay calm and observe.


💬 Enter a sentence to analyze sentiment (or type 'exit' to stop): exit
👋 Exiting sentiment analysis.
